<a href="https://colab.research.google.com/github/cdario79/AnalisiPilotiF1/blob/main/Analisi_dei_Piloti_del_Mondiale_di_Formula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the 'requests' library. This library is used to make HTTP requests,
# which allow your program to interact with web services and download data from websites.
import requests

# Import the 'csv' library. This library provides tools to work with CSV (Comma Separated Values) files,
# allowing you to read and write data in this common tabular format.
import csv

In [2]:
# This dictionary defines the points awarded to drivers based on their finishing position in a race.
# For example, the driver finishing in 1st place gets 10 points, 2nd place gets 8 points, and so on.
POINTS_BY_POSITION = {
    1: 10,
    2: 8,
    3: 6,
    4: 5,
    5: 4,
    6: 3,
    7: 2,
    8: 1
}

In [3]:
def read_data():
    """
    Reads data from the CSV file and returns it as a list of dictionaries.

    Each dictionary in the list represents a row in the CSV file,
    with keys corresponding to the column headers and values corresponding to the cell values.

    Returns:
        list: A list of dictionaries containing the data from the CSV file.
               Returns an empty list if there is an error fetching or processing the data.
    """
    try:
        # Define the URL of the CSV file to download.
        url = "https://raw.githubusercontent.com/cdario79/AnalisiPilotiF1/refs/heads/main/formula1_data.csv"

        # Send a GET request to the URL to download the file.
        response = requests.get(url)

        # If the request was unsuccessful, raise an exception.
        response.raise_for_status()

        # Create a CSV reader object that treats the first row as headers.
        reader = csv.DictReader(response.text.splitlines())

        # Return the data as a list of dictionaries.
        return list(reader)

    # Handle errors that may occur during data fetching or processing.
    except requests.exceptions.RequestException as e:
        # Print an error message if there's a problem with the request.
        print(f"Error fetching data: {e}")
        return []  # Return an empty list in case of error
    except (ValueError, KeyError) as e:
        # Print an error message if there's a problem processing the data.
        print(f"Error processing data: {e}")
        return []  # Return an empty list in case of error
    except Exception as e:
        # Print a generic error message for unexpected errors.
        print(f"An unexpected error occurred: {e}")
        return []  # Return an empty list in case of error

In [4]:
def analyze_driver(driver_name):
    """
    Analyzes the performance of a driver based on the provided dataset.

    Args:
        driver_name (str): The name of the driver to analyze.

    Returns:
        list: A list containing the total points, number of wins, and number of podiums for the driver.
              Returns an empty list if the driver is not found or an error occurs.
    """
    # Read the data from the CSV file.
    data = read_data()

    # Initialize variables to store the driver's statistics.
    total_points = 0
    wins = 0
    podiums = 0

    # Iterate through each row (race result) in the data.
    for row in data:
        # Check if the current row matches the driver we're analyzing.
        if row['Driver'] == driver_name:
            # Get the driver's finishing position in the race.
            position = int(row['Position'])

            # Get the points earned for the driver's finishing position.
            # If the position is not found in the POINTS_BY_POSITION dictionary (e.g., position > 8),
            # it defaults to 0 points, ensuring drivers outside the points positions are handled correctly.
            points = POINTS_BY_POSITION.get(position, 0)
            # Add the points to the driver's total points.
            total_points += points

            # Check if the driver won the race (position 1).
            if position == 1:
                # If so, increment wins and podiums.
                wins += 1
                podiums += 1
            # Check if the driver finished in 2nd or 3rd place.
            elif position == 2 or position == 3:
                # If so, increment podiums.
                podiums += 1

    # Return the driver's statistics as a list.
    return [total_points, wins, podiums]

In [5]:
# Example usage: Analyze the performance of a specific driver (e.g., "Hamilton").
driver_data = analyze_driver("Hamilton")
# If driver_data is not empty (meaning the driver was found), print the results.
if driver_data:
    # Print the driver's total points, wins, and podium finishes.
    print(f"Points: {driver_data[0]}, Wins: {driver_data[1]}, Podiums: {driver_data[2]}")

Points: 98, Wins: 5, Podiums: 10


In [ ]:
def save_driver_standings(standings, filename="Drivers_Standings_2008.txt"):
    """
    Saves the driver standings to a text file.

    Args:
        standings (dict): A dictionary containing the driver standings.
        filename (str, optional): The name of the file to save the standings to.
                                  Defaults to "Drivers_Standings_2008.txt".
    """
    try:
        with open(filename, "w") as file:
            file.write("Drivers Standings 2008 Formula 1\n")
            for driver, points in standings.items():
                file.write(f"{driver}: {points}\n")
    except OSError as e:
        print(f"Error saving driver standings to file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
def generate_driver_standings():
    """
    Generates and saves the final driver standings for the 2008 F1 season.

    This function reads race data, calculates total points for each driver,
    sorts them by points, and saves the standings to a text file.

    Returns:
        dict: A dictionary containing the driver standings,
              where keys are driver names and values are their total points.
    """
    # 1. Read race data from the CSV file.
    data = read_data()

    # 2. Create an empty dictionary to store driver points.
    driver_points = {}

    # 3. Get a list of all unique drivers in the dataset.
    all_drivers = list(set(row['Driver'] for row in data))

    # 4. Iterate through each driver in the list.
    for driver in all_drivers:
        # a. Analyze the driver's performance to get their total points.
        # We only need the total points, so we use '_' for wins and podiums.
        total_points, _, _ = analyze_driver(driver)
        # b. Store the driver's total points in the dictionary.
        driver_points[driver] = total_points

    # 5. Sort the drivers by their total points in descending order.
    sorted_standings = dict(sorted(driver_points.items(), key=lambda item: item[1], reverse=True))

    # 6. Save the driver standings to a text file.
    save_driver_standings(sorted_standings)

    # 7. Return the sorted driver standings dictionary.
    return sorted_standings

In [ ]:
# Example usage: Generate and print the driver standings.
driver_standings = generate_driver_standings()

# Print a header for the standings.
print("Drivers Standings 2008 Formula 1")

# Iterate through the driver standings dictionary.
for driver, points in driver_standings.items():
    # Print each driver's name and their total points.
    print(f"{driver}: {points}")

Drivers Standings 2008 Formula 1
Hamilton: 98
Massa: 97
Kubica: 75
Raikkonen: 75
Alonso: 61
Heidfeld: 60
Kovalainen: 53
Vettel: 35
Trulli: 31
Glock: 25


In [ ]:
def generate_constructor_standings():
    """
    Generates the constructor standings for the 2008 F1 season.

    This function reads race data, calculates total points for each constructor (team),
    sorts them by points, and returns the standings as a dictionary.

    Returns:
        dict: A dictionary containing the constructor standings,
              where keys are constructor names and values are their total points.
    """
    # 1. Read the race data from the CSV file.
    data = read_data()

    # 2. Create an empty dictionary to store constructor points.
    constructor_points = {}

    # 3. Iterate through each race result (row) in the data.
    for row in data:
        # a. Get the constructor (team) name from the current row.
        constructor = row['Team']
        # b. Get the driver's finishing position in this race.
        position = int(row['Position'])
        # c. Get the points earned for that position using POINTS_BY_POSITION (defaults to 0 if not found).
        points = POINTS_BY_POSITION.get(position, 0)
        # d. Add the points to the constructor's total, initializing to 0 if it's the first time we see this constructor.
        constructor_points[constructor] = constructor_points.get(constructor, 0) + points

    # 4. Sort the constructors by their total points in descending order.
    sorted_standings = dict(sorted(constructor_points.items(), key=lambda item: item[1], reverse=True))

    # 5. Return the sorted constructor standings.
    return sorted_standings

In [ ]:
# Example usage: Generate and print the constructor standings.
constructor_standings = generate_constructor_standings()

# Print a header for the standings.
print("Constructor Standings 2008 Formula 1")

# Iterate through the constructor standings dictionary.
for constructor, points in constructor_standings.items():
    # Print each constructor's name and their total points.
    print(f"{constructor}: {points}")

Constructor Standings 2008 Formula 1
Ferrari: 172
McLaren: 151
BMW: 135
Renault: 61
Toyota: 56
Toro Rosso: 35
